# Pythonic

프로그래밍에서 관용구(idiom)는 특정 작업을 수행하기 위해 코드를 작성하는 특별한 방법이다.  
매번 동일한 구조를 반복하고 따르는 것이 일반적이다.  
이는 언어와 무관한 고차원의 개념으로 코드로 즉시 변환되지 않는 디자인 패턴과는 다르다.  
    
관용구는 코드이므로 언어에 따라 다르며, 실제 코딩으로 변환된다.  
관용구를 따른 코드를 관용적이라고 부르며, Python에서는 Python스럽다(Pythonic)이라고 한다.

## 인덱스와 슬라이스

일반적으로 프로그래밍에서 array에 접근할 때, index를 0~n까지의 정수를 사용한다.  
하지만 Python에서는 음수를 통해서 접근할 수 있다.

In [1]:
my_numbers = (4, 5, 3, 9)
print(my_numbers[-1])
print(my_numbers[-3])

9
5


Slice르 사용하여 특정 구간의 요소를 구할 수도 있다.  
Slice의 `시작 인덱스`는 **포함**, `끝 인덱스`는 **제외**하고 선택한 구간의 값을 가져온다는 것에 유의하자.  
시작, 끝 또는 간격 파라미터 중 하나를 제외할 수 있으며, 이런 경우에는 아래의 결과처럼, 시퀸스의 처음 또는 끝에서부터 동작한다.  

`Slice를 사용한 경우, return 값과 original의 값의 관계는 deep copy인가 shallow copy인가?`

In [4]:
my_numbers = (1, 1, 2, 3, 5, 8, 13, 21)
print(my_numbers[2:5])

(2, 3, 5)


In [1]:
print(my_numbers[:3])
print(my_numbers[3:])
print(my_numbers[::]) # this case return value is tuple as copy from original tuple
print(my_numbers[1:7:2])

NameError: name 'my_numbers' is not defined

In [2]:
a = [[1,2,3], [4,5,6]]
print(a)

b = a[0]
b[0] = 5

print(a)
print(b)

[[1, 2, 3], [4, 5, 6]]
[[5, 2, 3], [4, 5, 6]]
[5, 2, 3]


위의 모든 예제에서 시퀸스에 간격을 전달할 때, 실제로는 Slice를 전달하는 것과 같다.  
slice는 파이썬 내장 객체로 직접 빌드하여 전달할 수도 있다.

mutable이냐 아니냐에 따라서, shallow copy, deep copy 바뀜 (케바케)

python은 call by object reference

In [ ]:
interval = slice(1, 7, 2)
print(my_numbers[interval])

interval = slice(None, 3)
print(my_numbers[interval] == my_numbers[:3])

### indexing & slice result is shallow copy or deep copy?

In [ ]:
my_numbers = [1, 1, 2, 3, 5, 8, 13, 21]

# slicing list
test1 = my_numbers[:3]
print(test1)

# change value in slicing object
test1[0] = 90
print(test1)
print(my_numbers, end="\n\n")

test1 = my_numbers[3:]
print(test1)
test1[0] = 90
print(test1)
print(my_numbers, end="\n\n")

test1 = my_numbers[::]
print(test1)
test1[0] = 90
print(test1)
print(my_numbers, end="\n\n")

test1 = my_numbers[1:7:2]
print(test1)
test1[0] = 90
print(test1)
print(my_numbers, end="\n\n")

**`슬라이싱의 결과는 Deep Copy다`**(x)

## 자체 시퀸스 생성

위에서 설명한 기능`(?)`은 `__getitem__`이라는 매직 메소드 덕분에 동작한다.  
이는 `myobject[key]`와 같은 형태를 사용할 때, 호출되는 메소드로 key에 해당하는 대괄호 안의 값을 파라미터로 전달한다.  
    
특히 시퀸스는 `__getitem__`과 `__len__`을 구현하는 객체이므로 반복이 가능하다.  
`list`, `tuple`, `str`은 표준 라이브러리에 있는 시퀸스 객체의 예이다.
    
해당 챕터에서는 `시퀸스`나 `이터레이블`객체를 만들지 않고, 키로 객체의 특정 요소를 가져오는 방법에 대해 다룬다.  
    
클래스가 표준 라이브러리 객체를 감싸는 래퍼인 경우 기본 객체에 가능한 많은 동작을 위임할 수 있다.  
즉 클래스가 리스트의 래퍼인 경우 리스트의 동일한 메소드를 호출하여 호환성을 유지할 수 있다.

### 캡슐화 방식을 사용한 리스트 랩핑

In [ ]:
class Items(object):
    def __init__(self, *values):
        self._values = list(values)
        
    def __len__(self):
        return len(self._values)
    
    def __getitem__(self, item):
        return self._values.__getitem__(item)

- 상속을 이용하는 방법도 있다
    - 상속을 이용하고싶다면, `collections.UserLIst` 부모 클래스를 상속받아야한다.
    - 이는 나중에 다룬다.
    
     
만약 래퍼도 아니고 내장 객체를 사용하지도 않는 경우는 자신만의 시퀸스를 구현할 수 있다.  
이때는 다음 사항에 유의해야한다.  
(`Python3에는 기본 class가 내장 객체를 기본적으로 상속받는 것으로 알고있는데, 이를 강제로 변경할 수 있나?`)
> 위의 이야기가 아니라, 이미 기존에 있는 래퍼방식을 쓰던가,   
> 상속받는 방식을 쓰는게 아니라, 순수하게 자신의 시퀸스 객체를 정의할 때, 이야기
    
    
    
- 범위로 인덱싱하는 결과는 해당 클래스와 같은 타입의 인스턴스여야 한다.
- slice에 의해 제공된 범위는 파이썬이 하는 것처럼 마지막 요소는 제외해야한다.  
    
<br/>
    
1. "범위로 인덱싱하는 결과는 해당 클래스와 같은 타입의 인스턴스여야 한다." 의 예시 `range`  
`interval`을 지정하여 range를 호출하면 선택한 범위의 값을 생성하는 방법을 알고 있는 이터러블 객체를 반환한다.  
range의 간격을 지정하면 당연하게도, 리스트가 아닌 새로운 range를 얻게 된다.  
```python
>> range(1, 100)[25:50]
   range(26, 51)
```
`list`의 `interval`을 지정해서 받은 `return` 값이 `str`이거나 `tuple`이라면 얼마나 황당한가?  
<br/>
2. slice에 의해 제공된 범위는 파이썬이 하는 것처럼 마지막 요소는 제외해야한다.  
이는 일관성에 대한 문제이다. python3 내장 slice가 마지막 요소를 제외하고 있으며, 사용자가 이에 익숙하다면,  
제공된 커스텀 객체가 이렇게 작동하지 않는다면, 사용자가 이를 혼동하고 사용하기 어려울 수 있다.
    

## Python3에는 기본 class가 내장 객체를 기본적으로 상속받는 것으로 알고있는데, 이를 강제로 변경할 수 있나?

In [3]:
print(dir(object))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [4]:
class test:
    def __init__(self):
        pass
    
class test1():
    def __init__(self):
        pass
    
class test2(object):
    def __init__(self):
        pass
    
a = test()
b = test1()
c = test2()

print(dir(a), end="\n\n")

print(dir(b), end="\n\n")

print(dir(c), end="\n\n")


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']



## Python2의 결과

```python
class test:
    def __init__(self):
        pass
    
class test1():
    def __init__(self):
        pass
    
class test2(object):
    def __init__(self):
        pass
    
a = test()
b = test1()
c = test2()

print(dir(a))
print

print(dir(b))
print

print(dir(c))
print
```

```python
>> python2 test.py
>> ['__doc__', '__init__', '__module__']
>> 
>> ['__doc__', '__init__', '__module__']
>> 
>> ['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']
>> 
```

**`모든 클래스는 기본적으로 object를 상속받는다.`**